In [1]:
import scipp as sc
import scippneutron as scn
from ess import loki, sans
from scippneutron.tof.conversions import elastic_wavelength, elastic_Q, beamline

In [2]:
def to_bin_edges(d, dim):
    centers = d.coords[dim].copy()
    del d.coords[dim]
    first = 1.5*centers[dim, 0] - 0.5*centers[dim, 1]
    last = 1.5*centers[dim, -1] - 0.5*centers[dim, -2]
    bulk = 0.5 * (centers[dim, 1:] + centers[dim, :-1])
    edges = sc.concat([first, bulk], dim)
    edges = sc.concat([edges, last], dim)
    d.coords[dim] = edges

In [3]:
def get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr):
      
    inv_l3 = (l_collimation + l2) / (l_collimation * l2)
    
    # Terms in Mildner and Carpenter equation.
    # See https://docs.mantidproject.org/nightly/algorithms/TOFSANSResolutionByPixel-v1.html
    a1 = (r1/l_collimation)*(r1/l_collimation) * 3.0
    a2 = (r2*inv_l3)*(r2*inv_l3) * 3.0
    a3 = (dr/l2) * (dr/l2) 
    
    sc.to_html(a1)
    wav_independent = (4.0 * sc.constants.pi / 12) * (a1 + a2 + a3) 
    return wav_independent, a1, a2, a3
    

In [4]:
def generate_gaussian(sigma, num_points):
    """
    Generates gaussian function based on supplied sigma
    """
    import scipy.stats as stats
    xpoints = sc.linspace('x', -3*sigma, 3*sigma, num=num_points)
    pdf_result = stats.norm.pdf(xpoints.values,loc=0,scale=sigma)

In [5]:
def get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2):

     
    #sigModerator = Moderator time spread (microseconds) as afunction of wavelength (Angstroms)
    sig_moderator = moderatorValue * 3.9560 / (1000.0 * (l1 + l2));
    sig_moderator.unit = sc.units.angstrom
    
    #TODO: replace with q from cooridnate transformations 
    theta = scn.two_theta(data) 
    q_sq = 4.0 * sc.constants.pi * sc.sin(0.5*theta) * sc.reciprocal(lam)
    q_sq *= q_sq
    
    std_dev_lam_sq = q_sq 
    std_dev_lam_sq *= (d_lam * d_lam)/12 + sig_moderator * sig_moderator
    std_dev_lam_sq *= sc.reciprocal(lam * lam)
    
    std_dev_lam_independent = sc.DataArray((wav_independent * sc.reciprocal(lam * lam) ),
                    coords={'wavelength':std_dev_lam_sq.coords['wavelength']}) 
   
    dq_sq = sc.sqrt(std_dev_lam_independent + std_dev_lam_sq)
    return dq_sq   


In [6]:
def q_resolution(wavelength_bins, moderator, data, q):
    
    l_collimation = 4.0*sc.units.m
    r2 = 0.004*sc.units.m
    r1 = 0.01*sc.units.m
    dr = 0.008*sc.units.m
    
    d_lam = wavelength_bins['wavelength', 1:] - wavelength_bins['wavelength', :-1] # bin widths
    lam = 0.5 * (wavelength_bins['wavelength', 1:] + wavelength_bins['wavelength', :-1]) # bin centres
    lam = sc.midpoints(x)
    l2 = scn.L2(data)
    l1 = scn.L1(data)
    
    a1, a2, a3, wav_independent = get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr)
    
    to_bin_edges(moderator, 'wavelength')
    moderatorValue = sc.rebin(moderator, 'wavelength', wavelength_bins)
    
    dq_sq = get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2)
    
    return a1, a2, a3, dq_sq 

In [7]:
# Estimate qresolution function

wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')

path = 'SANS2D_data'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
moderator = loki.io.load_rkh_wav(filename=f'{path}/{moderator_file}')
to_bin_edges(moderator, 'wavelength')

#q resolution params
l_collimation = 4.0*sc.units.m
r2 = 0.004*sc.units.m
r1 = 0.01*sc.units.m
dr = 0.008*sc.units.m

# Using only one-fourth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Sample measurement
sample = loki.io.load_sans2d(filename=f'{path}/SANS2D00063114.nxs',
                             spectrum_size=spectrum_size)

#d - data is normalized and reduced data

#dq_sq = q_resolution(wavelength_bins, moderator, sample, l_collimation, r1, r2, dr)

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


In [15]:
from scipp.constants import m_n, h
def two_theta(gravity, wavelength, incident_beam, scattered_beam):
    # Arbitrary internal convention: beam=z, gravity=y
    g = sc.norm(gravity)
    L2 = sc.norm(scattered_beam)
    y = sc.dot(scattered_beam, gravity) / g
    n = sc.cross(incident_beam, gravity)
    n /= sc.norm(n)
    x = sc.dot(scattered_beam, n)
    wavelength = sc.to_unit(wavelength, "m", copy=False)
    drop = g * m_n ** 2 / (2 * h ** 2) * wavelength ** 2 * L2 ** 2
    return sc.asin(sc.sqrt(x ** 2 + (y + drop) ** 2) / L2)

In [16]:
from scippneutron.tof.conversions import elastic_wavelength, elastic_Q, beamline

q_with_gravity = {**beamline(scatter=True),
                  **elastic_Q("tof")}
q_with_gravity["two_theta"] = two_theta

In [17]:
data = sample.copy(deep=False)
data.coords["gravity"] = sans.i_of_q.gravity_vector()
data_q = data.transform_coords("Q", graph=q_with_gravity)

In [18]:
data_q

<scipp.DataArray>
Dimensions: Sizes[spectrum:61440, Q:1, ]
Coordinates:
  Q                         float64           [1/Å]  (spectrum, Q [bin-edge])  [1312.2, 0.0649932, ..., 1312.2, 0.0648066]
  spectrum                    int32        <no unit>  (spectrum)  [9, 10, ..., 61447, 61448]
Data:
                          DataArrayView        <no unit>  (spectrum, Q)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  Q                         float64           [1/Å]  (event)  []
  pulse_time              datetime64             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []
Attributes:
  tof                       float64            [µs]  (event)  []
  two_theta                 float64            [rad]  (event)  []
  wavelength                float64             [Å]  (event)  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  Q                         float64           [1/Å]  (event)  []
  pulse_time              datetime64             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []
Attributes:
  tof                       float64            [µs]  (event)  []
  two_theta                 float64            [rad]  (event)  []
  wavelength                float64             [Å]  (event)  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  Q                         float64           [1/Å]  (event)  []
  pulse_time              datetime64             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []
Attributes:
  tof                       float64            [µs]  (event)  []
  two_theta                 float64            [rad]  (event)  []
  wavelength                float64             [Å]  (event)  []

, <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  Q                         float64           [1/Å]  (event)  [0.227518]
  pulse_time              datetime64             [ns]  (event)  [2020-02-12T20:53:37.686950683]
Data:
                            float32         [counts]  (event)  [1]  [1]
Attributes:
  tof                       float64            [µs]  (event)  [28523.6]
  two_theta                 float64            [rad]  (event)  [0.175264]
  wavelength                float64             [Å]  (event)  [4.83395]

]
Attributes:
  Changer                 DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000]
Data:
                             string  [dimensionless]  (time)  ["IT", "JT"]

]
  Det_Temp_FLB            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [16.8, 16.8, 16.8]

]
  Det_Temp_FLT            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Det_Temp_FRB            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Det_Temp_FRT            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 20

In [8]:
a1, a2, a3, dq_sq = q_resolution(wavelength_bins, moderator, sample, sample_q.coords['Q'])

TypeError: q_resolution() takes 4 positional arguments but 7 were given

In [20]:
lam = 0.5 * (wavelength_bins['wavelength', 1:] + wavelength_bins['wavelength', :-1]) # bin centres


In [21]:
lam

<scipp.Variable> (wavelength: 140)    float64             [Å]  [2.05, 2.15, ..., 15.85, 15.95]